In [45]:
import pandas as pd
import pandas as pd
import pandas as pd
from complexcgr import CGR
from pathlib import Path
# Instantiate class CGR
cgr = CGR()

def get_last_name(file_path):
    return Path(file_path).stem

def preprocessing(snp_list):
    # Step1: merge SNPs of all samples
    with open(snp_list, "r") as f:
        sample_files = f.read().splitlines()

    try:
        # Read the data from the first sample
        first_data = pd.read_csv(sample_files[0], sep=" ", header=None)
        first_data.columns = ["genomic_position", "REF", get_last_name(sample_files[0])]
        # Drop the REF column
        first_data = first_data.drop(columns=["REF"])

        # Start with first_data as the merged data
        merge_data = first_data

        # add other samples and merge them one by one
        for sample in sample_files[1:]:
            try:
                pd.read_csv(sample, sep=" ", header=None)
                print(sample)
                sample_data = pd.read_csv(sample, sep=" ", header=None)
                column_name = get_last_name(sample)
                sample_data.columns = ["genomic_position", "REF", column_name]
                # Drop the REF column
                sample_data = sample_data.drop(columns=["REF"])
                # merge data on genomic positions
                merge_data = merge_data.merge(sample_data, on="genomic_position", how="outer")
            except:
                continue

        merge_data.set_index("genomic_position", inplace=True)
        # Tranpose the df (reverse) to have samples as rows and genomic positions as columns
        merge_data = merge_data.T

        # Move the return statement outside the loop, after all samples have been processed and merged.
        return merge_data

    except Exception as e:
        print("Error:", str(e))
        return None


# Call the preprocessing function and store the result in the 'df' variable
df = preprocessing("../data/snp.lists")
#     merge_data = merge_data.fillna("N")
#     merge_data = merge_data.replace({"A": 1, "G": 2, "C": 3, "T": 4, "N": 0})
#     merge_data = merge_data.rename(columns={"V1": "POS"})
#     tinput = merge_data.T
#     tinput.columns = tinput.iloc[0]
#     tinput = tinput.iloc[1:]
#     value = len(sample_files) // 5 * 4
#     tinput4 = tinput[tinput["Num"] < value]
#     tinput5 = tinput4.T
#     tinput5.to_csv("snp_input.csv", index=False)

df=preprocessing("../data/snp.lists")
df
#     # FCGR Encoding using kaos package
#     data = pd.read_csv("snp_input.csv")
#     data = data.replace({0: "N", 1: "A", 2: "G", 3: "C", 4: "T"})

#     for i in range(900):
#         cgr_list = cgr(data[str(i)], seq_base=["N", "A", "G", "C", "T"], res=200)
#         cgr_matrix = cgr_list["matrix"]
#         cgr_matrix.to_csv(sample_files[i], index=False)


../data/SRR10502184.snp
../data/SRR12539197.snp
../data/SRR12539198.snp
../data/SRR12539199.snp
../data/SRR12539200.snp
../data/SRR12539202.snp
../data/SRR12539203.snp
../data/SRR12539204.snp
../data/SRR12539205.snp
../data/SRR12539206.snp
../data/SRR12539207.snp
../data/SRR15017103.snp
../data/SRR15279127.snp
../data/SRR15279128.snp
../data/SRR15282887.snp
../data/SRR15323717.snp
../data/SRR15323719.snp
../data/SRR15323721.snp
../data/SRR15323722.snp
../data/SRR15323723.snp
../data/SRR15323724.snp
../data/SRR15323725.snp
../data/SRR15323727.snp
../data/SRR15323728.snp
../data/SRR15323729.snp
../data/SRR15323731.snp
../data/SRR15323732.snp
../data/SRR15323733.snp
../data/SRR15323734.snp
../data/SRR15323809.snp
../data/SRR15323810.snp
../data/SRR15323811.snp
../data/SRR15323812.snp
../data/SRR15323813.snp
../data/SRR15369286.snp
../data/SRR15369287.snp
../data/SRR15369290.snp
../data/SRR15369291.snp
../data/SRR15369294.snp
../data/SRR15369295.snp
../data/SRR15369296.snp
../data/SRR15369

genomic_position,15,27,78,480,504,534,646,681,942,951,...,4854168,4854222,4854536,4854873,4855307,4855379,4856474,4856802,4856930,4857309
SRR10502163,T,A,A,G,T,C,T,C,G,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SRR10502184,T,A,A,G,T,C,T,C,G,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SRR12539197,NaN,NaN,NaN,G,NaN,C,T,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SRR12539198,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SRR12539199,NaN,NaN,NaN,G,NaN,C,T,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SRR12539200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SRR12539202,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SRR12539203,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SRR12539204,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SRR12539205,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Fillna with N

In [46]:
df=df.fillna("N")

In [47]:
df=df.reset_index()
df=df.rename(columns={"index":"sra_accession"})
df.head()

genomic_position,sra_accession,15,27,78,480,504,534,646,681,942,...,4854168,4854222,4854536,4854873,4855307,4855379,4856474,4856802,4856930,4857309
0,SRR10502163,T,A,A,G,T,C,T,C,G,...,N,N,N,N,N,N,N,N,N,N
1,SRR10502184,T,A,A,G,T,C,T,C,G,...,N,N,N,N,N,N,N,N,N,N
2,SRR12539197,N,N,N,G,N,C,T,N,N,...,N,N,N,N,N,N,N,N,N,N
3,SRR12539198,N,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,N,N,N,N
4,SRR12539199,N,N,N,G,N,C,T,N,N,...,N,N,N,N,N,N,N,N,N,N


In [48]:
df_bio=pd.read_table("../accessions/kenya/kenya.tsv")
df_bio=df_bio[["BioSample","Isolation source"]]
df_bio

,BioSample,Isolation source
0,SAMEA2080497,blood
1,SAMN03576958,feces swine
2,SAMN03576962,feces swine
3,SAMN03576963,feces swine
4,SAMN03576956,feces swine
...,...,...
272,SAMN20456392,Kibera
273,SAMN20456390,Kibera
274,SAMN20456385,Kibera
275,SAMN20456395,Kibera


In [49]:
df_sra=pd.read_table("../pipeline/run-2023-08-11-salmonellaml/sra_kenya_biosamples.txt",header=None)
df_sra.columns=["Biosample","sra_accession"]
df_sra

,Biosample,sra_accession
0,SAMN20456381,SRR15403520
1,SAMN20456395,SRR15369306
2,SAMN20456385,SRR15369305
3,SAMN20456390,SRR15369303
4,SAMN20456392,SRR15369302
...,...,...
129,SAMN03576959,SRR2163255
130,SAMN03576956,SRR2163254
131,SAMN03576963,SRR2163247
132,SAMN03576962,SRR2163246


In [50]:
# Concatenate the DataFrames along columns
# result_df = pd.concat([df1, df2, df3], axis=1)

In [56]:
df_sra_biosample=pd.concat([df_bio,df_sra], axis=1)
df_sra_biosample.dropna(inplace=True)
df_all=pd.concat([df_sra_biosample,df],axis=1)
df_all

,BioSample,Isolation source,Biosample,sra_accession,sra_accession,15,27,78,480,504,...,4854168,4854222,4854536,4854873,4855307,4855379,4856474,4856802,4856930,4857309
0,SAMEA2080497,blood,SAMN20456381,SRR15403520,SRR10502163,T,A,A,G,T,...,N,N,N,N,N,N,N,N,N,N
1,SAMN03576958,feces swine,SAMN20456395,SRR15369306,SRR10502184,T,A,A,G,T,...,N,N,N,N,N,N,N,N,N,N
2,SAMN03576962,feces swine,SAMN20456385,SRR15369305,SRR12539197,N,N,N,G,N,...,N,N,N,N,N,N,N,N,N,N
3,SAMN03576963,feces swine,SAMN20456390,SRR15369303,SRR12539198,N,N,N,N,N,...,N,N,N,N,N,N,N,N,N,N
4,SAMN03576956,feces swine,SAMN20456392,SRR15369302,SRR12539199,N,N,N,G,N,...,N,N,N,N,N,N,N,N,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,SAMN13906352,feces,SAMN03576959,SRR2163255,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
130,SAMN13906353,blood,SAMN03576956,SRR2163254,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
131,SAMN13906351,feces,SAMN03576963,SRR2163247,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132,SAMN13906349,feces,SAMN03576962,SRR2163246,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
# ! pip install sklearn

In [53]:
# df.columns
# df.to_csv("../data/processed_data.csv")

In [54]:
df.head()

genomic_position,sra_accession,15,27,78,480,504,534,646,681,942,...,4854168,4854222,4854536,4854873,4855307,4855379,4856474,4856802,4856930,4857309
0,SRR10502163,T,A,A,G,T,C,T,C,G,...,N,N,N,N,N,N,N,N,N,N
1,SRR10502184,T,A,A,G,T,C,T,C,G,...,N,N,N,N,N,N,N,N,N,N
2,SRR12539197,N,N,N,G,N,C,T,N,N,...,N,N,N,N,N,N,N,N,N,N
3,SRR12539198,N,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,N,N,N,N
4,SRR12539199,N,N,N,G,N,C,T,N,N,...,N,N,N,N,N,N,N,N,N,N


In [55]:
# One hot encoding

In [37]:
# Replicate for trial 


# Create a new dataset with 500 rows
# Replicate the data 5 times (total 500 rows)
for _ in range(6):
    replicated_data = pd.concat([df, df], ignore_index=True)


In [38]:
replicated_data

,Biosample,sra_accession
0,SAMN20456381,SRR15403520
1,SAMN20456395,SRR15369306
2,SAMN20456385,SRR15369305
3,SAMN20456390,SRR15369303
4,SAMN20456392,SRR15369302
...,...,...
263,SAMN03576959,SRR2163255
264,SAMN03576956,SRR2163254
265,SAMN03576963,SRR2163247
266,SAMN03576962,SRR2163246


In [39]:

# Step 2: Remove constant columns
non_constant_cols = replicated_data.columns[replicated_data.nunique() > 1]
replicated_data = replicated_data[non_constant_cols]
replicated_data.reindex()

,Biosample,sra_accession
0,SAMN20456381,SRR15403520
1,SAMN20456395,SRR15369306
2,SAMN20456385,SRR15369305
3,SAMN20456390,SRR15369303
4,SAMN20456392,SRR15369302
...,...,...
263,SAMN03576959,SRR2163255
264,SAMN03576956,SRR2163254
265,SAMN03576963,SRR2163247
266,SAMN03576962,SRR2163246


In [40]:
# Step 2: Perform one-hot encoding for all categorical columns
replicated_data_encoded = pd.get_dummies(df)
replicated_data_encoded

,Biosample_SAMN02843543,Biosample_SAMN02844775,Biosample_SAMN03576951,Biosample_SAMN03576952,Biosample_SAMN03576953,Biosample_SAMN03576954,Biosample_SAMN03576955,Biosample_SAMN03576956,Biosample_SAMN03576958,Biosample_SAMN03576959,...,sra_accession_SRR3747672,sra_accession_SRR3924533,sra_accession_SRR3924536,sra_accession_SRR4123985,sra_accession_SRR4123986,sra_accession_SRR4243040,sra_accession_SRR4733853,sra_accession_SRR4733857,sra_accession_SRR5972858,sra_accession_SRR6958628
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
130,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
131,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
132,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [117]:
# Step 3: Scale the data
scaler = MinMaxScaler()  # or StandardScaler()
scaled_data = scaler.fit_transform(replicated_data)

# Convert the scaled numpy array back to a DataFrame with the original columns
scaled_data_df = pd.DataFrame(scaled_data, columns=non_constant_cols)
scaled_data_df

NameError: name 'MinMaxScaler' is not defined

In [158]:
from sklearn.preprocessing import MinMaxScaler


ModuleNotFoundError: No module named 'sklearn'

In [154]:
! pip install sklearn

Defaulting to user installation because normal site-packages is not writeable
